In [ ]:
'''
@Title   :  基于ChatGLM2-6B+LoRA在普惠金融政策数据集上进行微调
@Time    :   2023/7/30 
@Author  :   王佳晨

'''

In [1]:
import subprocess
import os
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//10.0.2.207'), PosixPath('17606')}
  warn(msg)
/root/miniconda3/lib/python3

In [2]:
os.getcwd()

'/root/autodl-tmp/ChatGLM-Tuning'

# 对文本进行tokenize,加快训练速度

In [3]:
!python tokenize_dataset_rows.py \
    --jsonl_path data/puhui227_prepared.jsonl \
    --save_path data/puhui \
    --chatglm_path THUDM/chatglm2-6b \
    --version v1                 


Found cached dataset generator (/root/.cache/huggingface/datasets/generator/default-75c4361790b22033/0.0.0)


# 模型训练

In [4]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = AutoModel.from_pretrained("THUDM/chatglm2-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False  

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [6]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [7]:
import datasets

dataset_path = "data/puhui"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

In [8]:
from transformers import Trainer, HfArgumentParser


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss
    
def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu") for k, v in model.named_parameters() if v.requires_grad
    }
    torch.save(saved_params, path)

In [11]:
training_args = TrainingArguments(
    "output",
    fp16 =False,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-3,
    max_steps=600,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
50,2.547200
100,2.615700
150,2.691400
200,2.547200
250,2.441700
300,2.111300
350,2.132900
400,2.105400
450,2.155700
500,1.667100


/root/miniconda3/lib/python3.8/site-packages/torch/utils/checkpoint.py:391: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=600, training_loss=2.193267822265625, metrics={'train_runtime': 313.0753, 'train_samples_per_second': 1.916, 'train_steps_per_second': 1.916, 'total_flos': 5044623235448832.0, 'train_loss': 2.193267822265625, 'epoch': 2.64})

In [12]:
save_tunable_parameters(
        model, os.path.join("output", "chatglm2-lora.pt")
    )

In [15]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
torch.set_default_tensor_type(torch.cuda.HalfTensor)
peft_path = "output/chatglm2-lora.pt"
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

model = model.eval()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/__init__.py:615: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


# 微调效果

In [17]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。


In [18]:
response, history = model.chat(tokenizer, "列举一些普惠金融发展政策", history=[])
print(response)

普惠金融发展政策如下:

1. 《推进普惠金融发展规划(2016-2020年)》(国办发[2015]74号)

2. 《关于支持金融机构进一步改进小型微型企业金融服务(修订)》

3. 《关于积极发展普惠金融的指导意见》(银发[2015]25号)

4. 《关于普惠金融统计工作的指导意见》(银发[2015]26号)

5. 《普惠金融发展规划(2016-2020年)》(中国人民银行、中国银行保险监督管理委员会、国家发展改革委、财政部、工业和信息化部、财政部、证监会、保监会、国家外汇管理局2015年11月28日公布)


In [19]:
response, history = model.chat(tokenizer, "那请你概括一下《关于积极发展普惠金融的指导意见》并说出文件有什么需要特别注意的地方", history=[])
print(response)

《关于积极发展普惠金融的指导意见》是2015年4月29日中国人民银行等十部委联合发布的一份文件，旨在推动普惠金融业务健康发展，助力经济增长。普惠金融是指金融机构通过创新金融产品和服务方式，将金融服务更加普及和便捷，以满足小微企业、农民、城镇低收入人群、贫困人群等弱势群体的金融服务需求。

在推动普惠金融发展过程中，需要注意以下几点：

1. 强化金融基础设施建设，提升金融服务的普及率。发展普惠金融需要建立良好的金融生态环境，这需要建立完善的基础设施，如网点、网络、支付系统等，并不断提升服务水平，确保金融服务的普及率逐年提高。

2. 丰富普惠金融产品和服务，满足不同人群需求。发展普惠金融需要针对不同人群的需求，提供多样化的金融产品和服务，如贷款、保险、理财等，以满足小微企业、农民、城镇低收入人群、贫困人群等弱势群体的需求。

3. 创新金融模式，丰富金融供给。发展普惠金融需要不断探索新的金融模式，如网络银行、移动支付、贷款贴现等，丰富金融供给，提高服务效率。

4. 强化监管和引导，规范市场秩序。发展普惠金融需要强化监管和引导，规范市场秩序，打击非法金融活动，保障金融消费者的合法权益，维护金融市场秩序。

5. 加强政策支持，营造良好环境。发展普惠金融需要政策支持，营造良好的市场环境。政府应当加大对普惠金融的政策支持力度，推动金融机构加大对普惠金融的投入，引导金融市场健康发展。


In [35]:
response, history = model.chat(tokenizer, "《普惠金融发展规划(2016-2020年)》", history=[])
print(response)

《普惠金融发展规划（2016-2020年）》是为深入贯彻落实党中央、国务院关于金融支持实体经济发展的决策部署，落实党的十八大和十八届三中、四中、五中全会精神，按照“十三五”时期推进供给侧结构性改革要求，按照“普惠性、可获得性、灵活性、可持续性”四大核心要素，坚持“政府引导、市场主导”的普惠金融发展理念，健全激励机制，拓宽发展空间，完善普惠金融服务体系，全面提高金融服务实体经济能力，实现普惠金融可持续发展，为经济社会发展提供新动能、新支撑，实现“三农”和支小支微、精准扶贫精准脱贫、小微企业金融服务覆盖率明显提高，使每个市场主体在金融服务中受益，使每个普通消费者都有更好的金融获得感，使金融更好地服务实体经济，在全面建成小康社会基础上，到2020年，普惠金融服务覆盖率保持在较高水平，普惠金融服务重点领域覆盖率明显提高，每个县至少有1家以上具备条件的金融机构提供基础金融服务，每个县至少有1家以上具备条件的金融机构提供小微企业金融服务。

一、发展基础

（一）发展现状。当前，我国金融体系基本健全，但金融资源相对不足，结构不均衡，发展可持续性有待提高。基础金融服务普及率不断提高，但部分区域、领域和人群的金融服务需求仍然难以满足。小微企业、农民、残疾人、老年人等特殊群体和区域性贫困地区、欠发达地区仍然存在金融服务难点的制约。

（二）发展前景。随着经济发展和人民群众生活改善，人民群众对金融服务的需求日益增长，多层次、广覆盖、便捷化的金融服务将得到更为广阔的市场需求，未来一段时期我国普惠金融发展前景广阔。同时，新一代信息技术、大数据、云计算、区块链等金融科技的发展，将推动金融服务的创新与转型，为普惠金融发展提供新机遇。

二、发展目标

（一）总体目标。未来五年，基本实现全国每个县至少有1家以上具备条件的金融机构提供基础金融服务，每个县至少有1家以上具备条件的金融机构提供小微企业金融服务。小微企业贷款平均利率明显下降，小微企业贷款不良率明显下降。基础金融服务覆盖率保持在90%以上，每个县至少有1家以上具备条件的金融机构提供基础金融服务，每个县至少有1家以上具备条件的金融机构提供小微企业金融服务。普惠金融重点领域覆盖率明显提高，每个县至少有1家以上具备条件的金融机构提供助农金融服务。基本实现新纳入扶贫政策范围的有劳动能力且有稳定收入来源的贫困家庭至少拥有1种方便快捷的扶贫扶贫方式。

In [23]:
response, history = model.chat(tokenizer, "总结最近几年普惠金融发展状况", history=[])
print(response)

普惠金融是指将金融服务更加广泛、深入地覆盖到小微企业、农民、农村居民、贫困人口等弱势群体,具有更强的社会责任感和服务意识。近年来,我国普惠金融发展取得了一系列显著成效,主要表现在以下几个方面:

1. 普惠金融总量增长迅速。银行业金融机构截至2021年6月30日,单户授信总额1000万元及以下(含)小微企业贷款余额31.76万亿元,同比增长32.8%。截至2021年6月30日,全国小微企业贷款余额20.97万亿元,同比增长32.9%。

2. 银行业金融机构改进服务方式,提升服务水平。银行业金融机构在推进普惠金融数字化转型方面取得了一定成效,如通过“两民”信贷业务,改进服务方式,截至2021年6月30日,该业务余额11.17万亿元,同比增长40.2%。

3. 金融科技创新助力普惠金融发展。金融科技创新不断涌现,如蚂蚁金服推出了“三维风险定价系统”,降低小微企业贷款风险定价成本。同时,民营银行、网络银行等新型金融机构快速发展,截至2021年6月30日,民营银行数量达到21家,同比增长77%。

4. 政府出台了一系列扶持政策,推进普惠金融发展。政府出台了一系列扶持政策,如“两增两控”信贷政策,对小微企业贷款增速、户数、申贷获得率有明确要求。

5. 普惠金融在服务疫情防控和经济复苏中发挥了积极的作用。疫情期间,银行业金融机构对小微企业实施了优惠利率、免费服务、延长服务期限等优惠政策,为小微企业提供了支持。

当前普惠金融发展面临一些挑战,如对小微企业信用风险的判断、服务小微企业的能力、技术支撑、法律合规以及监管政策完善等。


In [22]:
response, history = model.chat(tokenizer, "中央如何推进小微企业金融服务建设", history=[])
print(response)

中央对于小微企业金融服务建设一直高度重视，通过多种途径推进小微企业金融服务建设。

一是加大对小微企业金融服务的顶层设计。党中央、国务院对小微企业金融服务作出了全面部署，出台了一系列文件，如《关于引导和规范金融机构切实改进小型微型企业金融服务的补充通知》、《关于支持商业银行进一步改进小型微型企业金融服务的补充通知》等，明确了不同金融机构的定位，强化内部管理和风险控制，规范小微企业金融服务。同时，还要求银行业金融机构加大对小微企业授信尽职免责的制度建设。

二是推动银行业金融机构创新小微企业金融服务方式。鼓励银行业金融机构在总结历史经验的基础上，针对小微企业融资特点，创新服务模式，改进风险管理手段，如在服务模式上，支持小微企业主将信用贷款和抵押担保相结合，推广“抵押登记、信用发放”的“纯信用”模式；在风险管理手段上，进一步改进内部评级、授信审批、贷款发放等流程，注重信用风险、抵押风险和担保风险的动态监测和预警，合理控制贷款风险，改进内部流程，提高效率。

三是引导金融机构加大对小微企业的有效信贷支持力度。引导金融机构根据小微企业生产经营周期特点，合理确定贷款期限，适当调整贷款到期还款方式，鼓励采用分期还款、循环贷款、融资租赁等方式，以缓解小微企业的资金周转压力。

四是强化对小微企业金融服务的组织保障。推动银行业金融机构建立健全小微企业金融服务专营机构，对小微企业贷款实行单独核算，单列考核指标，单独实施考核，单线管理风险，单独设置客服人员，单线进行贷后管理，建立小微企业专职管理人员队伍。对小微企业授信、贷后管理、资金流动、财务报表等全流程实施专业化管理。

五是鼓励社会力量积极参与小微企业金融服务建设。鼓励银行业金融机构与第三方机构合作，拓宽小微企业金融服务渠道，增加小微企业融资选择。鼓励银行、保险、证券、担保、融资担保、融资租赁、典当行的社会组织、企业法人、自然人等与银行业金融机构合作，在符合法律法规和监管政策前提下，有序开展小微企业金融服务的合作。


In [30]:
response, history = model.chat(tokenizer, "高新技术企业纳税政策", history=[])
print(response)

高新技术企业（简称“高企”）是指在《高新技术企业认定管理办法》（以下简称《认定办法》）规定的高新技术领域内，具备独立法人资格、注册成立满2年，并符合一系列技术、财务等条件的企业。对于高新技术企业，我国采取了一系列税收优惠政策，帮助企业减轻负担，促进其健康发展。

主要税收优惠政策如下：

1. 企业所得税优惠政策：
高企可以享受企业所得税减按15%税率征收的优惠政策。

2. 增值税优惠政策：
高企可以享受增值税一般纳税人资格的认定，使得企业可以享受简化纳税申报等税收优惠政策。

3. 企业所得税附加减半征收：
对高新技术企业，可按现行政策享受企业所得税定期减半征收企业所得税的优惠政策。

4. 投资优惠：
鼓励企业加大研发投入，促进产业结构升级，鼓励银行、证券、保险等金融机构为高新技术企业提供投资、融资、担保等服务。

5. 人才税收优惠政策：
对高新技术企业引进的高层次人才及其夫妻、未成年子女，可按照一定比例发放其工资薪金、年终奖金等收入，作为个人及其家庭主要生活来源，需要缴纳个人所得税的，可享受税收优惠政策。

6. 研发费用税前加计扣除政策：
高企发生的研发费用，按照规定在企业所得税应纳税所得额中可以据实扣除。

7. 进口税收优惠政策：
高企可以享受进口税收优惠政策，降低其研发成本。

8. 出口退税政策：
高企可以享受出口退税政策，降低其出口成本。

高企税收优惠政策会根据不同地区、不同行业以及不同企业类型有所不同，建议您查阅国家相关部门发布的《高新技术企业认定管理办法》和《企业所得税优惠政策事项办理办法》，以了解您企业所符合的高新技术企业税收优惠政策。
